In [11]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from sklearn.metrics import mean_absolute_error, mean_squared_error
import mlflow

class LSTMPredictor:
    def __init__(self, n_steps=3, n_features=1, n_units=50, n_epochs=10, batch_size=32, loss='mse', optimizer='adam'):
        self.n_steps = n_steps
        self.n_features = n_features
        self.n_units = n_units
        self.n_epochs = n_epochs
        self.batch_size = batch_size
        self.loss = loss
        self.optimizer = optimizer
        self.model = None
    
    def _create_model(self):
        model = Sequential()
        model.add(LSTM(self.n_units, activation='relu', input_shape=(self.n_steps, self.n_features)))
        model.add(Dense(1))
        model.compile(optimizer=self.optimizer, loss=self.loss)
        return model
    
    def train(self, X_train, y_train, X_val=None, y_val=None):
        with mlflow.start_run() as run:
            # Log parameters
            mlflow.log_params({
                "n_steps": self.n_steps,
                "n_features": self.n_features,
                "n_units": self.n_units,
                "n_epochs": self.n_epochs,
                "batch_size": self.batch_size,
                "loss": self.loss,
                "optimizer": self.optimizer
            })

            # Create model
            self.model = self._create_model()

            # Train model
            history = self.model.fit(
                X_train, y_train, 
                epochs=self.n_epochs, 
                batch_size=self.batch_size, 
                validation_data=(X_val, y_val) if X_val is not None else None,
                verbose=0
            )

            # Log metrics
            mlflow.log_metrics({
                "train_loss": history.history['loss'][-1],
                "val_loss": history.history['val_loss'][-1] if X_val is not None else None
            })

            # Save model
            mlflow.keras.log_model(self.model, "lstm_model")

    def predict(self, X_test):
        y_pred = self.model.predict(X_test)
        return y_pred.flatten()

    def evaluate(self, X_test, y_test):
        y_pred = self.predict(X_test)
        mae = mean_absolute_error(y_test, y_pred)
        mse = mean_squared_error(y_test, y_pred)
        with mlflow.start_run() as run:
            mlflow.log_metrics({
                "mae": mae,
                "mse": mse
            })
        return mae, mse


In [13]:
from datetime import datetime
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# leitura dos dados
data = pd.read_csv('dados.csv')

# normalização dos dados
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data[['CPU Load (%)', 'Memory RAM Usage (%)', 'Disk Usage (%)']].values)

# separação em treino e teste
train_size = int(len(scaled_data) * 0.7)
train_data = scaled_data[:train_size]
test_data = scaled_data[train_size:]

In [14]:
# preparação dos dados para a rede LSTM
timesteps = 3
X_train, y_train = LSTMPredictor.prepare_data(train_data, timesteps)
X_test, y_test = LSTMPredictor.prepare_data(test_data, timesteps)

AttributeError: type object 'LSTMPredictor' has no attribute 'prepare_data'

In [ ]:
# criação e treinamento do modelo LSTM
lstm = LSTMPredictor(input_shape=(X_train.shape[1], X_train.shape[2]), units=64, learning_rate=0.001)
lstm.set_tracking_uri("http://localhost:5000")
lstm.set_experiment("lstm-experiment")
lstm.train(X_train, y_train, epochs=50, batch_size=32)

# avaliação do modelo
y_pred = lstm.predict(X_test)
mse, mae, r2 = lstm.evaluate(X_test, y_test)

# log das métricas no MLflow
now = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
with mlflow.start_run(run_name=f"lstm-run-{now}") as run:
    # log dos hiperparâmetros
    mlflow.log_param("input_shape", lstm.input_shape)
    mlflow.log_param("units", lstm.units)
    mlflow.log_param("learning_rate", lstm.learning_rate)
    mlflow.log_param("epochs", 50)
    mlflow.log_param("batch_size", 32)
    
    # log das métricas de avaliação
    mlflow.log_metric("mse", mse)
    mlflow.log_metric("mae", mae)
    mlflow.log_metric("r2", r2)
    
    # registro do modelo no MLflow
    mlflow.keras.log_model(lstm.model, "lstm-model")